In [20]:
N_ROUNDS = 11

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import dataframe_image as dfi
from attacks import helper
import os
from pathlib import Path
pd.set_option('display.max_columns', None)

fmnist_b = pd.read_csv('data/wandb_export_2023-07-12_baseline_fashionmnist_f1.csv')
mnist_b = pd.read_csv('data/wandb_export_2023-07-12_baseline_mnist_f1.csv')
cifar10_b = pd.read_csv('data/wandb_export_2023-07-12_baseline_cifar10_f1.csv')

In [23]:
aggregation_list = ["Krum", "FedAvg", "TrimmedMean", "FlTrust", "Sentinel", "SentinelGlobal"]
aggregation_list.sort()
dataset_list = ["MNIST", "FashionMNIST", "CIFAR10"]
overview = pd.DataFrame(data=None, index=aggregation_list, columns=dataset_list, dtype=None, copy=None)

In [24]:
fmnist_f1 = fmnist_b[fmnist_b.columns.drop(list(fmnist_b.filter(regex='__MAX')))]
fmnist_f1 = fmnist_b[fmnist_b.columns.drop(list(fmnist_b.filter(regex='__MIN')))]

tmp = pd.DataFrame(index=range(0,N_ROUNDS), columns=aggregation_list)
for aggregator in aggregation_list:
    vals = fmnist_b.filter(regex=(f'{aggregator}_')).dropna()
    vals = vals.reset_index().iloc[:, 1]
    tmp[aggregator] = vals
    assert len(vals) == N_ROUNDS
tmp = tmp.reindex(sorted(tmp.columns), axis=1)
tmp = tmp.transpose()
overview['FashionMNIST'] = tmp[10]

In [25]:
mnist_f1 = mnist_b[mnist_b.columns.drop(list(mnist_b.filter(regex='__MAX')))]
mnist_f1 = mnist_f1[mnist_f1.columns.drop(list(mnist_f1.filter(regex='__MIN')))]

tmp = pd.DataFrame(index=range(0,N_ROUNDS), columns=aggregation_list)
for aggregator in aggregation_list:
    vals = mnist_f1.filter(regex=(f'{aggregator}_')).dropna()
    vals = vals.reset_index().iloc[:, 1]
    assert len(vals) == N_ROUNDS
    tmp[aggregator] = vals
tmp = tmp.reindex(sorted(tmp.columns), axis=1)
tmp = tmp.transpose()
overview['MNIST'] = tmp[10]

In [26]:
cifar10_f1 = cifar10_b[cifar10_b.columns.drop(list(cifar10_b.filter(regex='__MAX')))]
cifar10_f1 = cifar10_f1[cifar10_f1.columns.drop(list(cifar10_f1.filter(regex='__MIN')))]

tmp = pd.DataFrame(index=range(0,N_ROUNDS), columns=aggregation_list)
for aggregator in aggregation_list:
    vals = cifar10_f1.filter(regex=(f'{aggregator}_')).dropna()
    vals = vals.reset_index().iloc[:, 1]
    assert len(vals) == N_ROUNDS
    tmp[aggregator] = vals
tmp = tmp.reindex(sorted(tmp.columns), axis=1)
tmp = tmp.transpose()
overview['CIFAR10'] = tmp[10]

In [27]:
overview

overview = overview.sort_index()
overview

,MNIST,FashionMNIST,CIFAR10
FedAvg,0.952733,0.837768,0.764270
FlTrust,0.952309,0.837385,0.764245
Krum,0.935353,0.822143,0.671111
Sentinel,0.951335,0.833987,0.753640
SentinelGlobal,0.953639,0.832156,0.753487
TrimmedMean,0.951521,0.835301,0.762286


In [31]:
pd.set_option('colheader_justify', 'center')
overview_style = overview.style.set_caption(f'F1-Score\n')
#overview_style.highlight_max(props='font-weight: bold')
#tbl_style = overview_style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
#tbl_style.set_properties(**{'text-align': 'center'}).hide_index()
dfi.export(overview_style, "figures/baseline_overview_f1.png", dpi=300)